In [ ]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
#print(stopwords.words('english'))

In [3]:
import sys  
!{sys.executable} -m pip install contractions

  ERROR: Command errored out with exit status 1:
   command: 'D:\Anaconda3\envs\env_keras\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-1ncrds66\\pyahocorasick_0aef2f25e9fc49b6adfd7e52673cc450\\setup.py'"'"'; __file__='"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-1ncrds66\\pyahocorasick_0aef2f25e9fc49b6adfd7e52673cc450\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\user\AppData\Local\Temp\pip-wheel-9wzcputr'
       cwd: C:\Users\user\AppData\Local\Temp\pip-install-1ncrds66\pyahocorasick_0aef2f25e9fc49b6adfd7e52673cc450\
  Complete output (5 lines):
  running bdist_wheel
  running build
  running build_ext
  building 'ahocorasick' extension
  err


  Using cached contractions-0.1.73-py2.py3-none-any.whl (8.7 kB)
  Using cached textsearch-0.0.24-py2.py3-none-any.whl (7.6 kB)
  Using cached anyascii-0.3.2-py3-none-any.whl (289 kB)
  Using cached pyahocorasick-2.0.0.tar.gz (99 kB)
  Running setup.py clean for pyahocorasick
Failed to build pyahocorasick
    Running setup.py install for pyahocorasick: started
    Running setup.py install for pyahocorasick: finished with status 'error'


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from nltk.util import ngrams

data_path = 'data.txt'

input_texts = []
output_texts = []
with open(data_path) as f:
    lines = f.read().split('\n')
for line in lines[: min(600, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    output_texts.append(target_text)


import contractions
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    text = contractions.fix(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text_tokens = text.split()
    filtered_text = [word for word in text_tokens if word not in stop_words]
    text = ' '.join(filtered_text)
    # Lemmatize words
    # lemmatizer = WordNetLemmatizer()
    # text_tokens = text.split()
    # lemmatized_text = [lemmatizer.lemmatize(word) for word in text_tokens]
    # text = ' '.join(lemmatized_text)
    return text

# Preprocess input and output sequences
input_texts = [preprocess_text(text) for text in input_texts]
output_texts = [preprocess_text(text) for text in output_texts]

# Define the n-gram order
n = 2

# Generate n-gram sequences
input_ngram_sequences = []
output_ngram_sequences = []

for text in input_texts:
    ngrams_sequence = [' '.join(ngram) for ngram in ngrams(text.split(), n)]
    input_ngram_sequences.append(' '.join(ngrams_sequence))

for text in output_texts:
    ngrams_sequence = [' '.join(ngram) for ngram in ngrams(text.split(), n)]
    output_ngram_sequences.append(' '.join(ngrams_sequence))

# Configure the Tokenizer with n-grams support
tokenizer = Tokenizer(lower=True, filters='', split=' ')
tokenizer.fit_on_texts(input_ngram_sequences + output_ngram_sequences)

# Convert text sequences to integer sequences
input_sequences = tokenizer.texts_to_sequences(input_ngram_sequences)
output_sequences = tokenizer.texts_to_sequences(output_ngram_sequences)

# Define maximum sequence length
max_seq_length = max(len(seq) for seq in input_sequences + output_sequences)

# Pad sequences to the same length
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding='post')
output_sequences = pad_sequences(output_sequences, maxlen=max_seq_length, padding='post')

# Define vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Define encoder input, decoder input, and decoder output
encoder_input = input_sequences
decoder_input = np.zeros_like(output_sequences)
decoder_input[:, 1:] = output_sequences[:, :-1]
decoder_output = np.eye(vocab_size)[output_sequences]

# Define the Seq2Seq model
latent_dim = 128
dropout_rate = 0.2

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]
encoder_dropout = Dropout(dropout_rate)
encoder_dropout_output = encoder_dropout(state_h)

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dropout = Dropout(dropout_rate)
decoder_dropout_outputs = decoder_dropout(decoder_outputs)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_dropout_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Define K-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True)

# Define RLHF parameters
rl_epochs = 5  # Number of RL epochs
rl_learning_rate = 0.001  # RL learning rate
rl_batch_size = 64  # RL batch size
reinforce_reward = 1  # Reward value for positive reinforcement

for fold, (train_indices, val_indices) in enumerate(kf.split(input_sequences)):
    print(f'Fold {fold + 1}')
    x_train, x_val = input_sequences[train_indices], input_sequences[val_indices]
    y_train, y_val = decoder_output[train_indices], decoder_output[val_indices]
    decoder_input_train, decoder_input_val = decoder_input[train_indices], decoder_input[val_indices]

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=3)

    # Train the model using supervised learning
    history = model.fit([x_train, decoder_input_train], y_train,
                        validation_data=([x_val, decoder_input_val], y_val),
                        epochs=10, batch_size=64, callbacks=[early_stop])

    # Apply RLHF
    for rl_epoch in range(rl_epochs):
        # Generate translations using the current model
        predictions = model.predict([x_train, decoder_input_train])
        decoded_sequences = []
        for prediction in predictions:
            decoded_sequence = []
            for token in prediction:
                sampled_token_index = np.argmax(token)
                sampled_token = tokenizer.index_word.get(sampled_token_index, '')
                if sampled_token != '':
                    decoded_sequence.append(sampled_token)
            decoded_sequences.append(decoded_sequence)

        # Calculate rewards based on BLEU score or other metrics
        # rewards = calculate_rewards(decoded_sequences, ground_truth_sequences)

        # Prepare RL data by converting sequences to integer sequences
        rl_input_sequences = tokenizer.texts_to_sequences(decoded_sequences)
        rl_input_sequences = pad_sequences(rl_input_sequences, maxlen=max_seq_length, padding='post')

        # Train the model using RL
        model.optimizer.lr.assign(rl_learning_rate)
        model.fit([x_train, rl_input_sequences], y_train, epochs=1, batch_size=rl_batch_size)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate([x_val, decoder_input_val], y_val)
    print(test_loss)
    print(test_acc)



ModuleNotFoundError: No module named 'contractions'

In [ ]:
model.save("RL_seq2seq")

In [4]:
# Define a function to preprocess a single input text
def preprocess_input(text):
    # Implement the same preprocessing steps as in the training code
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = contractions.fix(text)
    stop_words = set(stopwords.words('english'))
    text_tokens = text.split()
    filtered_text = [word for word in text_tokens if word not in stop_words]
    text = ' '.join(filtered_text)
    return text

# Define a function to generate a response given an input text
def generate_response(input_text):
    preprocessed_text = preprocess_input(input_text)
    ngrams_sequence = [' '.join(ngram) for ngram in ngrams(preprocessed_text.split(), n)]
    input_ngram_sequence = ' '.join(ngrams_sequence)
    input_sequence = tokenizer.texts_to_sequences([input_ngram_sequence])
    padded_sequence = pad_sequences(input_sequence, maxlen=max_seq_length, padding='post')
    decoder_input = np.zeros((1, max_seq_length))
    decoder_input[:, 1:] = padded_sequence[:, :-1]
    predictions = model.predict([padded_sequence, decoder_input])
    decoded_sequence = []
    for token in predictions[0]:
        sampled_token_index = np.argmax(token)
        sampled_token = tokenizer.index_word.get(sampled_token_index, '')
        if sampled_token != '':
            decoded_sequence.append(sampled_token)
    response = ' '.join(decoded_sequence)
    return response

# Test the model on new input texts
input_texts_test = [
    "How are you?",
    "What is your name?",
    "Can you help me?",
    "Where are you from?",
    "What is the weather like today?",
    "what is dyslexia",
    "When were you born?",
    "it possible to identify dyslexia in a youngster who has hearing loss?"
]

for input_text in input_texts_test:
    response = generate_response(input_text)
    print("Input:", input_text)
    print("Response:", response)
    print()


1/1 [==============================] - 0s 349ms/step
Input: How are you?
Response: 

1/1 [==============================] - 0s 309ms/step
Input: What is your name?
Response: 

1/1 [==============================] - 0s 308ms/step
Input: Can you help me?
Response: 

1/1 [==============================] - 0s 308ms/step
Input: Where are you from?
Response: 

1/1 [==============================] - 0s 312ms/step
Input: What is the weather like today?
Response: chatbot chatbot chatbot

1/1 [==============================] - 0s 307ms/step
Input: what is dyslexia
Response: 

1/1 [==============================] - 0s 304ms/step
Input: When were you born?
Response: 

1/1 [==============================] - 0s 184ms/step
Input: it possible to identify dyslexia in a youngster who has hearing loss?
Response: assist help dyslexia dyslexia dyslexia dyslexia dyslexia dyslexia

